In [2]:
# !pip install pandas
# !pip install watermark
# !pip install seaborn
# !pip install biopython
# !pip install sklearn
import os
import re 
# from Bio import SeqIO
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np 
import watermark
import random 
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# !ls
# !unzip 'Training and testing datasets.zip'  -d './'

In [4]:
%load_ext watermark
%watermark
%watermark --iversion

Last updated: 2021-04-30T09:44:38.292813+08:00

Python implementation: CPython
Python version       : 3.7.3
IPython version      : 7.5.0

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 4.18.0-80.7.1.el8_0.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 52
Architecture: 64bit

numpy    : 1.20.2
watermark: 2.2.0
pandas   : 1.2.4
re       : 2.2.1



In [5]:
'''
DATA CLEAN
train set contains 462 files,2831 introns, all suffies of file is "TXT",the letters are  lowercase  letters.
test set contains 570 files, 2071 introns,the suffixes of file contains "TXT"and "txt",and the letters are capital letters.
'''

'\nDATA CLEAN\ntrain set contains 462 files, all suffies of file is "TXT",the letters are  lowercase  letters.\ntest set contains 570 files, the suffixes of file contains "TXT","txt",and the letters are capital letters.\n'

In [6]:
left=3
right=6 
signal_num = left+right
bases="ACGT"
lower_bases="acgt"
# noknown_bases="nksrywmbv" # manual counting

In [10]:
def mkdir(dirs):
    if not os.path.exists(dirs):
        os.makedirs(dirs)
    else:
        pass
mkdir("output")
mkdir("predict")

In [8]:
def loadFile(file_dir):
    '''
    Function: Read  All files in the Training Set Folder
    Parameter：file_dir
    Output: file_path,locus_list
    Attention: do not load non-fasta files!
    '''
    file_path = []
    file_locus_list = []
    count=0
    all_file = tqdm(os.listdir(file_dir), desc=f'LOADING {file_dir}')
    for file_name in all_file:
        count+=1
        suffix = re.findall("\.(.+$)",file_name)[-1].lower()
        # or  suffix = file_name.split(".")[1].lower()
        if suffix != "txt":
            continue
        path = f"{file_dir}/{file_name}"
        file_path.append(path)
    all_file.write(f"{count} Files Loading Finished!")
    all_file.close()
    print()
    return file_path,file_locus_list
train_file_path,train_file_locus =loadFile('Training Set')
test_file_path,test_file_locus =loadFile('Testing Set')
# print('LOADING File'.center(50,'*'))
# print(train_file_path)
# print(len(train_file_locus))

LOADING Testing Set: 100%|██████████| 571/571 [00:00<00:00, 406750.61it/s]

462 Files Loading Finished!

571 Files Loading Finished!



尝试使用bipython的模块，然而好像不太好用……还是用自己的函数吧

In [ ]:

# for seq_record in SeqIO.parse('Testing Set/ACU08131.TXT', "fasta"):
#     print(seq_record.id)
# #     print(type(seq_record.id))
# #     print(seq_record.seq)
#     print(str(seq_record.seq))

In [ ]:
原来不对序列编码，发现运行速度贼慢，希望用numpy速度会快点

In [ ]:
# function to convert a DNA sequence string to a numpy array
# converts to lower case, changes any non 'acgt' characters to 'n'
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array
# function to encode a DNA sequence string as an ordinal vector
# returns a numpy vector with a=0, c=1, g=2, t=3, n=4
def ordinal_encoder(my_array):
    label_encoder = LabelEncoder()
    label_encoder.fit(np.array(['a','c','g','t','z']))
    integer_encoded = label_encoder.transform(my_array)
#     print(integer_encoded)
#     float_encoded = integer_encoded.astype(float)
#     float_encoded[float_encoded == 0] = 0.25 # A
#     float_encoded[float_encoded == 1] = 0.50 # C
#     float_encoded[float_encoded == 2] = 0.75 # G
#     float_encoded[float_encoded == 3] = 1.00 # T
#     float_encoded[float_encoded == 4] = 0.00 # anything else, z
    return integer_encoded
def one_hot_encoder(my_array):
    label_encoder = LabelEncoder()
    label_encoder.fit(np.array(['a','c','g','t','z']))
    integer_encoded = label_encoder.transform(my_array)
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#     onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
x=getKmers(str(seq_record.seq), size=6)
mySeq = 'CATGGCCATCCCCCCCCGAGCGGGGGGGGGG'
mySeq2 = 'GATGGCCATCCCCGCCCGAGCGGGGGGGG'
mySeq3 = 'CATGGCCATCCCCGCCCGAGCGGGCGGGG'
for test_sequence in x:
    float_encoded = ordinal_encoder(string_to_array(test_sequence))
#     one_hot_matrix = one_hot_encoder(string_to_array(test_sequence))
    print(float_encoded)
#     print(one_hot_matrix)


In [7]:
##### 读取每个文件，抽取donor site 前3和后6共九个碱基 #####
def extract_donor_signal(file_path,set_folder):
    '''
    Parameter：train_file_path|test_file_path[dir]
    Output:file_donor_positions,file_acceptor_positions,file_donor_signals,donor_signal_all
    '''
    print(f'Extract {set_folder} Set donor signals'.center(50, '*'))
    file_donor_positions= []
    file_acceptor_positions= []
    file_donor_signals=[]
    file_acceptor_signals=[]
    all_donor_signal_str=[]
    all_acceptor_signal_str=[]
    file_length_list = []
    file_seq_list = []
    file_exons = []
    # positions=[]
    # print("Progressing：",end='')
    file_path = tqdm(file_path, desc=f'{set_folder} Progressing：')
    for file in file_path:
        f = open(file)
        _=f.readline() # pass the first line 
        second_line=f.readline()  
        # extract  donor and acceptor site
        donor_positions_list = re.findall(r'\d+\.\.(\d+)',second_line)[:-1] 
        donor_positions_list = [int(pos)+1 for pos in donor_positions_list] 
        acceptor_positions_list = re.findall(r'(\d+)\.\.\d+',second_line)[1:]
        acceptor_positions_list = [int(pos)-1 for pos in acceptor_positions_list]
        exon_positions_list = re.findall(r'(\d+)\.\.(\d+)',second_line)
        file_exons.append(exon_positions_list)
        file_donor_positions.append(donor_positions_list)
        file_acceptor_positions.append(acceptor_positions_list)
        seq = ''
        for line in f.readlines():
                seq += line.strip()
        # extract  donor site signal
        seq_length = len(seq)
        file_seq_list.append(seq)
        file_length_list.append(seq_length)
        donor_signal=[]
        for pos in donor_positions_list:
            # print(pos)
            if pos+right-1<=seq_length:
                signal_range = seq[pos-1-left:pos-1+right].lower() # Lowercase letters
            else:
                continue
            # seq_range = seq[pos-left:pos+right] # 有4个位点就是正好在序列最后， 所以需要剔除
            # donor_signal.append(list(seq_range))
            donor_signal.append(signal_range)
            all_donor_signal_str.append(signal_range)
        file_donor_signals.append(donor_signal)
        acceptor_signal=[]
        for pos in acceptor_positions_list:
            # print(pos)
            if pos-right>=0:
                signal_range = seq[pos-right:pos+left].lower() # Lowercase letters
            else:
                continue 
            acceptor_signal.append(signal_range)
            all_acceptor_signal_str.append(signal_range)
        file_acceptor_signals.append(acceptor_signal)
    print("Finished!")
    return file_donor_positions,file_donor_signals,all_donor_signal_str,file_acceptor_positions,\
file_acceptor_signals,all_acceptor_signal_str,file_length_list,file_seq_list,file_exons

#  str= "CDSjoin(28199..28271,28881..28988,34291..34586)"
#  exon_positions_list = re.findall(r'(\d+)\.\.(\d+)',str)
#  print(exon_positions_list)

train_file_donor_positions,train_file_donor_signals,train_donor_signal_all_str,\
train_file_acceptor_positions,train_file_acceptor_signals,train_acceptor_signal_all_str,\
train_file_length_list,train_file_seq_list,train_file_exons=extract_donor_signal(train_file_path,dir="Train")

test_file_donor_positions,test_file_donor_signals,test_donor_signal_all_str,\
test_file_acceptor_positions,test_file_acceptor_signals,test_acceptor_signal_all_str,\
test_file_length_list,test_file_seq_list,test_file_exons=extract_donor_signal(test_file_path,dir="Test")
# print(train_file_donor_positions)
# print(train_file_acceptor_positions)
# print(train_file_donor_signals[0])
# print(len(train_file_donor_signals)) # return the num of rows
# print(train_donor_signal_all_str)
# print(test_file_donor_positions)
# print(test_file_acceptor_positions)
# print(test_file_donor_signals[0])
# print(len(test_file_donor_signals)) # return the num of rows
# print(test_donor_signal_all_str)

Train Progressing：:  32%|███▏      | 148/462 [00:00<00:00, 1132.00it/s]

*********Extract Train Set donor signals**********


Test Progressing：:  33%|███▎      | 188/570 [00:00<00:00, 1874.65it/s]

Finished!
**********Extract Test Set donor signals**********


Test Progressing：: 100%|██████████| 570/570 [00:00<00:00, 1812.57it/s]

Finished!


In [8]:
# 才发现文本里不只是有ACGT，还有其他数值
def count_each_char(str_list):
    dict = {}
    str_list = tqdm(str_list, desc='Counting  Base Distribution:')
    for seq in str_list:
         for i  in seq:
            if i not in dict:
                dict[i] = 1
            else:
                dict[i] += 1
    return dict

base_dis_1 = count_each_char(train_file_seq_list)
base_dis_2 = count_each_char(train_donor_signal_all_str)
base_dis_3 = count_each_char(test_file_seq_list)
base_dis_4 = count_each_char(test_donor_signal_all_str)
print("Distribution of all bases in training set\n",base_dis_1)
print("Distribution of donor site bases in training  set\n",base_dis_2)
print("Distribution of all bases in testing set\n",base_dis_3)
print("Distribution of donor site  bases in testing set\n",base_dis_4)

Counting  Base Distribution:: 100%|██████████| 2079/2079 [00:00<00:00, 646401.63it/s]

Distribution of all bases in training set
 {'c': 1282733, 't': 1498203, 'a': 1436966, 'g': 1306260, 'n': 682, 'k': 28, 'b': 4, 's': 27, 'm': 16, 'r': 15, 'v': 2, 'y': 26, 'w': 14}
Distribution of donor site bases in training  set
 {'c': 2030, 'g': 8909, 't': 4709, 'a': 5781}
Distribution of all bases in testing set
 {'G': 702343, 'A': 736600, 'T': 762900, 'C': 689433, 'N': 862, 'Y': 3, 'R': 5, 'B': 1, 'S': 1, 'K': 1}
Distribution of donor site  bases in testing set
 {'c': 1691, 'a': 5114, 'g': 7730, 't': 4176}


In [9]:
def signal_to_csv(donor_signal_str,mode,dir="Train"):
    donor_signal_list=[list(line) for line in donor_signal_str]
    if mode=="donor" or mode=="nonDonor":
        col_name = list(range(-left,right))
    elif mode=="acceptor":
        col_name = list(range(-right+1,left+1))
    donorDf = pd.DataFrame(columns=col_name, data=donor_signal_list, index=None)
    print(donorDf)  # [2843 rows x 9 columns]
    donorDf.to_csv(f'output/{dir}_{mode}_signal_info.csv',index=None)
def save_donors_str(signal_str,filename="signal_str"):
    f = open(f'output/{filename}.txt', 'w')
    for donor in signal_str:
        donor = donor.lower()  # 存入文件时，把所有字符都规范成小写
        f.write(donor + '\n')
    f.close()
    print(f'save {filename} successful!')
signal_to_csv(train_donor_signal_all_str,dir="Train",mode="donor")
signal_to_csv(train_acceptor_signal_all_str,dir="Train",mode="acceptor")
signal_to_csv(test_donor_signal_all_str,dir="Test",mode="donor")
signal_to_csv(test_acceptor_signal_all_str,dir="Test",mode="acceptor")
save_donors_str(train_donor_signal_all_str,filename="Train_donor_signal_str")
save_donors_str(train_acceptor_signal_all_str,filename="Train_acceptor_signal_str")
save_donors_str(test_donor_signal_all_str,filename="Test_donor_signal_str")
save_donors_str(test_acceptor_signal_all_str,filename="Test_acceptor_signal_str")

     -3 -2 -1  0  1  2  3  4  5
0     c  g  g  g  t  a  t  g  t
1     g  c  a  g  t  g  a  g  t
2     a  a  t  g  t  a  a  g  t
3     c  a  g  g  t  a  a  g  a
4     a  a  g  g  t  g  a  g  t
...  .. .. .. .. .. .. .. .. ..
2376  a  a  g  g  t  a  a  g  a
2377  c  a  g  g  t  g  a  g  a
2378  a  a  g  g  t  a  a  g  g
2379  c  a  g  g  t  g  a  g  g
2380  t  c  g  g  t  g  a  g  t

[2381 rows x 9 columns]
     -5 -4 -3 -2 -1  0  1  2  3
0     c  c  c  c  a  g  g  a  t
1     t  t  c  t  a  g  t  g  c
2     t  c  a  c  a  g  a  t  t
3     g  g  a  c  a  g  g  g  c
4     t  t  t  c  a  g  a  t  c
...  .. .. .. .. .. .. .. .. ..
2376  c  t  a  a  a  g  a  a  a
2377  c  c  t  c  a  g  g  a  g
2378  c  c  c  c  a  g  g  a  g
2379  c  t  g  c  a  g  t  c  c
2380  c  a  t  t  a  g  a  a  g

[2381 rows x 9 columns]
     -3 -2 -1  0  1  2  3  4  5
0     c  a  g  g  t  t  g  g  t
1     a  g  g  g  t  g  a  g  t
2     c  g  g  g  t  a  t  g  t
3     c  g  g  g  t  g  a  g  t
4     a  t  g  g  t  g

In [10]:

def add_seq_to_csv(df_set_info,seq_list,dir='Training'):
    # 这里可以基于上一个添加
    df_set_info["Seq"] = seq_list
    df_set_info.to_csv(f'output/{dir}_set_info.csv',index=None)
    print(f"Created {dir}_set_info.csv !")
#     return df_set_info
def file_info_simple(file_path,file_locus,donor_positions,acceptor_positions,donor_signals,acceptor_signals,length_list,file_exons,dir='Training'):
    
    df_set_info = pd.DataFrame({'Path':file_path, 'Locus':file_locus,"Length":length_list,"Exon Num":[ len(exons)for exons  in file_exons],"Exon Location":file_exons,"Donor Site":donor_positions,\
                              "Acceptor Site":acceptor_positions,"Donor signals":donor_signals,"Acceptor signals":acceptor_signals})
    df_set_info.to_csv(f'output/{dir}_set_info(non-seq).csv',index=None)
    print(f"Created {dir}_set_info(non-seq).csv !")
    return df_set_info

train_set_info=file_info_simple(train_file_path,train_file_locus,train_file_donor_positions,\
                 train_file_acceptor_positions,train_file_donor_signals,train_file_acceptor_signals,\
                 train_file_length_list,train_file_exons,dir='Train')
test_set_info=file_info_simple(test_file_path,test_file_locus,test_file_donor_positions,\
                 test_file_acceptor_positions,test_file_donor_signals,test_file_acceptor_signals,
                 test_file_length_list,test_file_exons,dir='Test')
add_seq_to_csv(train_set_info,train_file_seq_list,dir='Train')
add_seq_to_csv(test_set_info,test_file_seq_list,dir='Test')

Created Train_set_info(non-seq).csv !
Created Test_set_info(non-seq).csv !
Created Train_set_info.csv !
Created Test_set_info.csv !


In [11]:

def cal_priorProbability(donor_signal) :
    priors = []
    count = []
    count= np.zeros((9,4),dtype=np.int32)
    priors = np.zeros((9,4),dtype=np.float32)
    # count[1][0]+1
    donor_signal = tqdm(donor_signal, desc='Calculating Prior Probability:')
    for donor in donor_signal:
        # print(donor)
        for j in range(len(donor)):
            pos = lower_bases.index(donor[j])
            count[j][pos] += 1
    for i in range(len(priors)):
        for j in range(4):
            priors[i][j] = count[i][j]/len(donor_signal)
    return priors
def prior_to_csv(priors,mode):
    pri_col_name = [f'p({base})' for base in bases ]
    pri_row_name= list(range(-left,right))
    
    priorDf= pd.DataFrame(index=pri_row_name,columns=pri_col_name, data=priors)
#     print(priorDf.isnull().sum()) #有四列缺失值
    print(priorDf)  # [2843 rows x 9 columns]
    priorDf.to_csv(f'output/prior_probability_(P{mode}).csv')
    
priors_p=cal_priorProbability(train_donor_signal_all_str)
prior_to_csv(priors_p,mode="+")
# print(priors)

Calculating Prior Probability:: 100%|██████████| 2381/2381 [00:00<00:00, 36281.67it/s]

        p(A)      p(C)      p(G)      p(T)
-3  0.328433  0.363713  0.188576  0.119278
-2  0.585468  0.137757  0.133557  0.143217
-1  0.094078  0.034019  0.788744  0.083158
 0  0.000000  0.000000  1.000000  0.000000
 1  0.000000  0.000000  0.000000  1.000000
 2  0.490130  0.027299  0.458211  0.024360
 3  0.713566  0.077278  0.117598  0.091558
 4  0.065099  0.049559  0.838303  0.047039
 5  0.151197  0.162957  0.216716  0.469131


In [12]:
#计算条件概率
def cal_conditionalProbability(donor_signal,priors):
    N,M=8,16
    count= np.zeros((N,M),dtype=np.int32)
    joint = np.zeros((N,M),dtype=np.float32)
    conditional = np.zeros((N,M),dtype=np.float32)
    # count
    donor_signal = tqdm(donor_signal, desc='Calculating Conditional Probability:')
    for donor in donor_signal:
        for j in range(N):
            base_list = [i+j for i in lower_bases for j in lower_bases ]
            pos=base_list.index(donor[j:j+2])
            count[j][pos] +=1
    #  joint
    joint= count/(len(donor_signal))
    ## conditional P(i+1,A|i,C) = P(A,C)/P(C) 第i位是C，第i+1位为A的概率，
    for i in range(8):
        for j in range(4):
            for m in range(4):
                if priors[i][j]:
                    conditional[i][4*j+m] = joint[i][4*j+m]/priors[i][j]
                else:
                    conditional[i][4*j+m] =0
    return conditional

def conditional_to_csv(conditional,mode):
    conditional_col_name = [f'p({i},{j})' for i in bases for j in bases ]
    conditional_row_name=list(range(-left+1,right))
    conditionalDf= pd.DataFrame(index=conditional_row_name,columns=conditional_col_name, data=conditional)
    # donorDf.isnull().sum() 有四列缺失值
    print(conditionalDf)  # [2843 rows x 9 columns]
    conditionalDf.to_csv(f'output/conditionalDf_probability(P{mode}).csv')
    
conditional = cal_conditionalProbability(train_donor_signal_all_str,priors_p)
conditional_to_csv(conditional,mode="+")

Calculating Conditional Probability:: 100%|██████████| 2381/2381 [00:00<00:00, 23352.58it/s]

      p(A,A)    p(A,C)    p(A,G)    p(A,T)    p(C,A)    p(C,C)    p(C,G)  \
-2  0.616368  0.097187  0.149616  0.136829  0.677829  0.123557  0.058891   
-1  0.073888  0.018651  0.854376  0.053085  0.189024  0.070122  0.524390   
 0  0.000000  0.000000  1.000000  0.000000  0.000000  0.000000  1.000000   
 1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 3  0.646101  0.095973  0.121680  0.136247  0.676923  0.030769  0.107692   
 4  0.057681  0.027075  0.882284  0.032961  0.163043  0.222826  0.429348   
 5  0.212903  0.135484  0.445161  0.206452  0.296610  0.288136  0.118644   

      p(C,T)    p(G,A)    p(G,C)    p(G,G)    p(G,T)    p(T,A)    p(T,C)  \
-2  0.139723  0.608018  0.167038  0.124722  0.100223  0.183099  0.246479   
-1  0.216463  0.147799  0.066038  0.707547  0.078616  0.035191  0.032258   
 0  0.000000  0.000000  0.000000  1.000000  0.000000  0.000000  0.000000   
 1  0.00000

In [13]:
# create 


# def create_nonDonor_rand(seqs_DNA, donor_locations, acceptor_locations,randnum=400):
#     '''
#     # 目前是暴力把所有非signal位点都导出，5510888 算了，太多了，但是又和PPT上的数值比较相近，我还是选它吧
#     可能会尝试取随机数，产生pos随机数，然后导出相应数量的假位点, 184800，但是这样各碱基背景概率会偏向相同
#     '''
#     nonDonors = []
#     file_num = tqdm(range(len(donor_locations)), desc='Creating Non Donor Signal Sequence:')
#     nonDonor_temp = []
#     for i in file_num:
#         # 每个文件循环
#         file_seq_DNA = seqs_DNA[i]
#         num = len(donor_locations[i])  # 寻找相应数量的假位点，我有三个就抽三个吗
#         length = len(file_seq_DNA)
#         donor_signals_start=[int(pos)-1-left for pos in donor_locations[i]]
# #         print(donor_locations[i])
#         acceptor_signals_start=[int(pos)-right for pos in acceptor_locations[i] if int(pos)-right>=0 ]
#         signals_start=sorted(donor_signals_start+acceptor_signals_start)
# #         print(signals_start)
#         count = 0
#         pos_list=[]
#         while count < randnum:
# #             print(count)
#             pos = random.randint(0,length-signal_num) 
#             if (pos not in signals_start) and (pos not in pos_list):
#                 nonDonor = file_seq_DNA[pos:pos + signal_num]
#                 for i in noknown_bases:
#                     if i in nonDonor:
#                         break
#                 else:
#                     nonDonor_temp.append(nonDonor)
#                     pos_list.append(pos)
#                     count +=1
#         # 把所有假位点生成之后，还继续生成？
# #         slice = random.sample(phonyDonor_temp, length)
# #         nonDonors.extend(slice)
#     print('Created Non Donor Signal Sequence successful!')
#     return nonDonor_temp
# 生成假donor片段

def create_nonDonor(seqs_DNA, donor_locations, acceptor_locations,randnum=10):
    '''
    # 目前是暴力把所有非signal位点都导出，5510888 算了，太多了
    可能会尝试取随机数，产生pos随机数，然后导出相应数量的假位点
    '''
    nonDonors = []
    file_num = tqdm(range(len(donor_locations)), desc='Creating Non Donor Signal Sequence:')
    nonDonor_temp = []
    for i in file_num:
        # 每个文件循环
#         print(i)
#         nonDonor_temp = []
        file_seq_DNA = seqs_DNA[i]
        num = len(donor_locations[i])  # 寻找相应数量的假位点，我有三个就抽三个吗
        length = len(file_seq_DNA)
        donor_signals_start=[int(pos)-1-left for pos in donor_locations[i]]
#         print(donor_locations[i])
        acceptor_signals_start=[int(pos)-right for pos in acceptor_locations[i] if int(pos)-right>=0 ]
        signals_start=sorted(donor_signals_start+acceptor_signals_start)
#         print(signals_start) 
        for index in range(length-signal_num+1):
            if index not in signals_start:
                nonDonor = file_seq_DNA[index:index + signal_num]
                for i in noknown_bases:
                    if i in nonDonor:
                        break
                else:
                    nonDonor_temp.append(nonDonor)
        # 把所有假位点生成之后，还继续生成？
#         slice = random.sample(phonyDonor_temp, length)
#         nonDonors.extend(slice)
    print('Created Non Donor Signal Sequence successful!')
    return nonDonor_temp

nonDonor_list=create_nonDonor(train_file_seq_list, train_file_donor_positions, train_file_acceptor_positions)
len(nonDonor_list)


Creating Non Donor Signal Sequence:: 100%|██████████| 462/462 [00:05<00:00, 88.03it/s]

Created Non Donor Signal Sequence successful!


5511802

可以看到导出了4592个非donor site siganl 序列（因为剔除了很多未知碱基吧?按道理来说这种方式不应该出现超万个的序列吗？？？？)  
好吧，我知道了，原来又是循环位置放错  
现在导出了5511802个  

In [14]:
count_each_char(nonDonor_list)

Counting  Base Distribution:: 100%|██████████| 5511802/5511802 [00:07<00:00, 694850.44it/s]


{'c': 11520141, 't': 13457006, 'a': 12904584, 'g': 11724487}

In [15]:
signal_to_csv(nonDonor_list,mode="nonDonor") # 5510888
priors_n=cal_priorProbability(nonDonor_list)
prior_to_csv(priors_n,mode="-")
conditional_n = cal_conditionalProbability(nonDonor_list,priors_n)
conditional_to_csv(conditional_n,mode="-")

        -3 -2 -1  0  1  2  3  4  5
0        c  t  t  t  a  a  t  t  t
1        t  t  t  a  a  t  t  t  t
2        t  t  a  a  t  t  t  t  a
3        t  a  a  t  t  t  t  a  t
4        a  a  t  t  t  t  a  t  c
...     .. .. .. .. .. .. .. .. ..
5511797  c  t  t  t  a  g  a  t  g
5511798  t  t  t  a  g  a  t  g  g
5511799  t  t  a  g  a  t  g  g  a
5511800  t  a  g  a  t  g  g  a  g
5511801  a  g  a  t  g  g  a  g  a

[5511802 rows x 9 columns]


Calculating Conditional Probability::   0%|          | 2393/5511802 [00:00<03:50, 23929.62it/s]

        p(A)      p(C)      p(G)      p(T)
-3  0.260176  0.232064  0.236535  0.271225
-2  0.260070  0.232180  0.236552  0.271198
-1  0.260208  0.232249  0.236201  0.271342
 0  0.260328  0.232092  0.236200  0.271381
 1  0.259927  0.232399  0.236639  0.271034
 2  0.260156  0.232376  0.236024  0.271444
 3  0.259963  0.232284  0.236368  0.271385
 4  0.260241  0.232256  0.236185  0.271318
 5  0.260195  0.232187  0.236456  0.271162


Calculating Conditional Probability:: 100%|██████████| 5511802/5511802 [03:50<00:00, 23919.36it/s]

      p(A,A)    p(A,C)    p(A,G)    p(A,T)    p(C,A)    p(C,C)    p(C,G)  \
-2  0.296746  0.187205  0.281931  0.234117  0.306976  0.294569  0.075583   
-1  0.297091  0.187342  0.281303  0.234265  0.307077  0.294593  0.075395   
 0  0.297018  0.187027  0.281816  0.234139  0.307245  0.294354  0.075511   
 1  0.296868  0.187233  0.281858  0.234041  0.306103  0.295006  0.075614   
 2  0.297431  0.187513  0.280660  0.234396  0.307051  0.294625  0.075501   
 3  0.296638  0.187263  0.281964  0.234135  0.307043  0.294606  0.075506   
 4  0.297221  0.187353  0.281156  0.234271  0.307098  0.294596  0.075449   
 5  0.296928  0.187163  0.281862  0.234047  0.307079  0.294574  0.075490   

      p(C,T)    p(G,A)    p(G,C)    p(G,G)    p(G,T)    p(T,A)    p(T,C)  \
-2  0.322872  0.257471  0.230251  0.293022  0.219256  0.187022  0.223623   
-1  0.322935  0.257576  0.230293  0.292836  0.219296  0.187008  0.223645   
 0  0.322891  0.257991  0.230343  0.292053  0.219613  0.187020  0.223537   
 1  0.32327

数据处理部分结束！😀😀😀😀😀😀😀😀😀😀😀😀😀